<a href="https://colab.research.google.com/github/Xindranil/LLMGraphTransformer/blob/main/Shift_Log_to_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install langchain langchain_community langchain_groq langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.9 MB/s eta 0:00:00


In [13]:
from langchain_groq import ChatGroq

from google.colab import userdata
groq_api_key= userdata.get('groq_api_key')
neo4j_password = userdata.get('Neo4j_password')

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key = groq_api_key
)

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader('Shift_Logs.txt')
documents = loader.load()
documents

[Document(metadata={'source': 'Shift_Logs.txt'}, page_content='Shift Log Entry 1:\nDate: 2024-12-28\nShift: Morning\nOperator: John Doe\nEquipment: Komatsu Truck #23\nIssues Observed:\n- High engine temperature warning.\n- Decreased hydraulic pressure.\nActions Taken:\n- Checked cooling system and added coolant.\n- Inspected hydraulic lines for leaks, no visible damage found.\nRecommendations:\n- Monitor engine temperature in the next shift.\n- Schedule detailed hydraulic system inspection.\n\nShift Log Entry 2:\nDate: 2024-12-28\nShift: Afternoon\nOperator: Jane Smith\nEquipment: Komatsu Truck #23\nIssues Observed:\n- Persistent high engine temperature.\n- Sluggish acceleration.\nActions Taken:\n- Reduced operational load to minimize stress on the engine.\n- Logged data for temperature and performance metrics.\nRecommendations:\n- Perform engine diagnostics during maintenance hours.\n\nShift Log Entry 3:\nDate: 2024-12-29\nShift: Night\nOperator: Mike Brown\nEquipment: Komatsu Truck #

In [27]:
!pip install neo4j
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph

URI = "neo4j+s://742cf5de.databases.neo4j.io"
AUTH = neo4j_password

graph = Neo4jGraph(url=URI,username="neo4j",password = AUTH)
driver = GraphDatabase.driver(URI, auth=("neo4j", AUTH))

In [34]:
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
nodes = graph_documents[0].nodes
releations = graph_documents[0].relationships
graph.add_graph_documents(graph_documents)

In [30]:
def results_to_string(results):
    result_strings = [
        ", ".join(f"{key}: {value}" for key, value in result.items())
        for result in results
    ]
    return "\n".join(result_strings)
def query (query):
  with driver.session() as session:
    results = session.run(query).data()
    results = results_to_string(results)
    return results

In [41]:
query("""MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m""")

"n: {'id': 'John Doe'}, r: ({'id': 'John Doe'}, 'OPERATOR', {'id': 'Komatsu Truck #23'}), m: {'id': 'Komatsu Truck #23'}\nn: {'id': 'Jane Smith'}, r: ({'id': 'Jane Smith'}, 'OPERATOR', {'id': 'Komatsu Truck #23'}), m: {'id': 'Komatsu Truck #23'}\nn: {'id': 'Mike Brown'}, r: ({'id': 'Mike Brown'}, 'OPERATOR', {'id': 'Komatsu Truck #23'}), m: {'id': 'Komatsu Truck #23'}\nn: {'id': 'Komatsu Truck #23'}, r: ({'id': 'Komatsu Truck #23'}, 'USED_ON', {'id': '2024-12-28'}), m: {'id': '2024-12-28'}\nn: {'id': 'Komatsu Truck #23'}, r: ({'id': 'Komatsu Truck #23'}, 'USED_ON', {'id': '2024-12-29'}), m: {'id': '2024-12-29'}\nn: {'id': '2024-12-28'}, r: None, m: None\nn: {'id': '2024-12-29'}, r: None, m: None"